In [ ]:
import numpy as np
import statsmodels.api as sm
from pandas.stats.api import ols
from sklearn.linear_model import LinearRegression

import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, PercentChange, SimpleMovingAverage, BusinessDaysSincePreviousEvent, BusinessDaysUntilNextEvent
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.factset import Fundamentals as FstFundamentals
# from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline import factors, filters, classifiers
import alphalens
import quantopian.pipeline.data.factset.estimates as fe


def signalize(df):
    return ((df.rank() - 0.5)/df.count()).replace(np.nan,0.5)

def transform(df, field, multiplier=1):
    return signalize(multiplier*df[field])

class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1
    window_safe = True
    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = close[-1] * shares[-1]

class LogMarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = np.log(close[-1] * shares[-1])

class Volatility(CustomFactor):  
    inputs = [USEquityPricing.close]  
    window_length = 25  
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0) * math.sqrt(252)

class Industry(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]

        
class IndustryGroup(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_group_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]


class Sector(CustomFactor):  
    inputs = [Fundamentals.morningstar_sector_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]

class Previous(CustomFactor):  
    # Returns value of input x trading days ago where x is the window_length  
    # Both the inputs and window_length must be specified as there are no defaults

    def compute(self, today, assets, out, inputs):  
        out[:] = inputs[0]


        
base_universe = QTradableStocksUS()

mkt_cap = MarketCap(mask = base_universe)
log_mkt_cap = LogMarketCap(mask = base_universe)
vol = Volatility(mask = base_universe)

industry = Industry(mask = base_universe)
industryGroup = IndustryGroup(mask = base_universe)
sector = Sector(mask = base_universe)

guidance_days_prev = BusinessDaysSincePreviousEvent(inputs=[fe.Guidance.slice('SALES', 'qf', 1).asof_date])  
guidance_days_next = BusinessDaysUntilNextEvent(inputs=[fe.Guidance.slice('SALES', 'qf', 1).asof_date])  


# f18 = f3.demean(mask = f13.notnan(), groupby=classifiers.fundamentals.Sector())


#Quarterly Historical EPS/SALES growth - Actual
# fq0_eps_act = fe.Actuals.slice('EPS', 'qf', 0).actual_value.latest
# fqn1_eps_act = fe.Actuals.slice('EPS', 'qf', -1).actual_value.latest
# fqn2_eps_act = fe.Actuals.slice('EPS', 'qf', -2).actual_value.latest
# fqn3_eps_act = fe.Actuals.slice('EPS', 'qf', -3).actual_value.latest
# fqn4_eps_act = fe.Actuals.slice('EPS', 'qf', -4).actual_value.latest
# fqn5_eps_act = fe.Actuals.slice('EPS', 'qf', -5).actual_value.latest
# fqn6_eps_act = fe.Actuals.slice('EPS', 'qf', -6).actual_value.latest
# fqn7_eps_act = fe.Actuals.slice('EPS', 'qf', -7).actual_value.latest
# fqn8_eps_act = fe.Actuals.slice('EPS', 'qf', -8).actual_value.latest

# fq0_cfps_act = fe.Actuals.slice('CFPS', 'qf', 0).actual_value.latest
# fqn1_cfps_act = fe.Actuals.slice('CFPS', 'qf', -1).actual_value.latest
# fqn2_cfps_act = fe.Actuals.slice('CFPS', 'qf', -2).actual_value.latest
# fqn3_cfps_act = fe.Actuals.slice('CFPS', 'qf', -3).actual_value.latest
# fqn4_cfps_act = fe.Actuals.slice('CFPS', 'qf', -4).actual_value.latest
# fqn5_cfps_act = fe.Actuals.slice('CFPS', 'qf', -5).actual_value.latest
# fqn6_cfps_act = fe.Actuals.slice('CFPS', 'qf', -6).actual_value.latest
# fqn7_cfps_act = fe.Actuals.slice('CFPS', 'qf', -7).actual_value.latest
# fqn8_cfps_act = fe.Actuals.slice('CFPS', 'qf', -8).actual_value.latest


fq0_sales_act = fe.Actuals.slice('SALES', 'qf', 0).actual_value.latest
fqn1_sales_act = fe.Actuals.slice('SALES', 'qf', -1).actual_value.latest
fqn2_sales_act = fe.Actuals.slice('SALES', 'qf', -2).actual_value.latest
fqn3_sales_act = fe.Actuals.slice('SALES', 'qf', -3).actual_value.latest
fqn4_sales_act = fe.Actuals.slice('SALES', 'qf', -4).actual_value.latest
fqn5_sales_act = fe.Actuals.slice('SALES', 'qf', -5).actual_value.latest
fqn6_sales_act = fe.Actuals.slice('SALES', 'qf', -6).actual_value.latest
fqn7_sales_act = fe.Actuals.slice('SALES', 'qf', -7).actual_value.latest
fqn8_sales_act = fe.Actuals.slice('SALES', 'qf', -8).actual_value.latest


fq1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 1).mean.latest
fq0_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', 0).mean.latest
fqn1_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -1).mean.latest
fqn2_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -2).mean.latest
fqn3_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -3).mean.latest
fqn4_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -4).mean.latest
fqn5_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -5).mean.latest
fqn6_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -6).mean.latest
fqn7_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -7).mean.latest
fqn8_sales_cons = fe.PeriodicConsensus.slice('SALES', 'qf', -8).mean.latest


#EPS Guidance
guid_eps_q1_high = fe.Guidance.slice('EPS', 'qf', 1).high.latest
guid_eps_q0_high = fe.Guidance.slice('EPS', 'qf', 0).high.latest
guid_eps_qn1_high = fe.Guidance.slice('EPS', 'qf', -1).high.latest
guid_eps_qn2_high = fe.Guidance.slice('EPS', 'qf', -2).high.latest
guid_eps_qn3_high = fe.Guidance.slice('EPS', 'qf', -3).high.latest
guid_eps_qn4_high = fe.Guidance.slice('EPS', 'qf', -4).high.latest
guid_eps_qn5_high = fe.Guidance.slice('EPS', 'qf', -5).high.latest
guid_eps_qn6_high = fe.Guidance.slice('EPS', 'qf', -6).high.latest
guid_eps_qn7_high = fe.Guidance.slice('EPS', 'qf', -7).high.latest
guid_eps_qn8_high = fe.Guidance.slice('EPS', 'qf', -8).high.latest

guid_eps_q1_low = fe.Guidance.slice('EPS', 'qf', 1).low.latest
guid_eps_q0_low = fe.Guidance.slice('EPS', 'qf', 0).low.latest
guid_eps_qn1_low = fe.Guidance.slice('EPS', 'qf', -1).low.latest
guid_eps_qn2_low = fe.Guidance.slice('EPS', 'qf', -2).low.latest
guid_eps_qn3_low = fe.Guidance.slice('EPS', 'qf', -3).low.latest
guid_eps_qn4_low = fe.Guidance.slice('EPS', 'qf', -4).low.latest
guid_eps_qn5_low = fe.Guidance.slice('EPS', 'qf', -5).low.latest
guid_eps_qn6_low = fe.Guidance.slice('EPS', 'qf', -6).low.latest
guid_eps_qn7_low = fe.Guidance.slice('EPS', 'qf', -7).low.latest
guid_eps_qn8_low = fe.Guidance.slice('EPS', 'qf', -8).low.latest

#Sales Guidance
guid_sales_q1_high = fe.Guidance.slice('SALES', 'qf', 1).high.latest
guid_sales_q0_high = fe.Guidance.slice('SALES', 'qf', 0).high.latest
guid_sales_qn1_high = fe.Guidance.slice('SALES', 'qf', -1).high.latest
guid_sales_qn2_high = fe.Guidance.slice('SALES', 'qf', -2).high.latest
guid_sales_qn3_high = fe.Guidance.slice('SALES', 'qf', -3).high.latest
guid_sales_qn4_high = fe.Guidance.slice('SALES', 'qf', -4).high.latest
guid_sales_qn5_high = fe.Guidance.slice('SALES', 'qf', -5).high.latest
guid_sales_qn6_high = fe.Guidance.slice('SALES', 'qf', -6).high.latest
guid_sales_qn7_high = fe.Guidance.slice('SALES', 'qf', -7).high.latest
guid_sales_qn8_high = fe.Guidance.slice('SALES', 'qf', -8).high.latest

guid_sales_q1_low = fe.Guidance.slice('SALES', 'qf', 1).low.latest
guid_sales_q0_low = fe.Guidance.slice('SALES', 'qf', 0).low.latest
guid_sales_qn1_low = fe.Guidance.slice('SALES', 'qf', -1).low.latest
guid_sales_qn2_low = fe.Guidance.slice('SALES', 'qf', -2).low.latest
guid_sales_qn3_low = fe.Guidance.slice('SALES', 'qf', -3).low.latest
guid_sales_qn4_low = fe.Guidance.slice('SALES', 'qf', -4).low.latest
guid_sales_qn5_low = fe.Guidance.slice('SALES', 'qf', -5).low.latest
guid_sales_qn6_low = fe.Guidance.slice('SALES', 'qf', -6).low.latest
guid_sales_qn7_low = fe.Guidance.slice('SALES', 'qf', -7).low.latest
guid_sales_qn8_low = fe.Guidance.slice('SALES', 'qf', -8).low.latest

pipe_my_alpha_factor = Pipeline(
    columns={
        'vol': vol,
        'log_mkt_cap': log_mkt_cap,
        'mkt_cap':mkt_cap,
        
# 'fq0_eps_act':fq0_eps_act,
# 'fqn1_eps_act':fqn1_eps_act,
# 'fqn2_eps_act':fqn2_eps_act,
# 'fqn3_eps_act':fqn3_eps_act,
# 'fqn4_eps_act':fqn4_eps_act,
# 'fqn5_eps_act':fqn5_eps_act,
# 'fqn6_eps_act':fqn6_eps_act,
# 'fqn7_eps_act':fqn7_eps_act,
# 'fqn8_eps_act':fqn8_eps_act,

# 'fq0_cfps_act':fq0_cfps_act,
# 'fqn1_cfps_act':fqn1_cfps_act,
# 'fqn2_cfps_act':fqn2_cfps_act,
# 'fqn3_cfps_act':fqn3_cfps_act,
# 'fqn4_cfps_act':fqn4_cfps_act,
# 'fqn5_cfps_act':fqn5_cfps_act,
# 'fqn6_cfps_act':fqn6_cfps_act,
# 'fqn7_cfps_act':fqn7_cfps_act,
# 'fqn8_cfps_act':fqn8_cfps_act,

        
'fq0_sales_act':fq0_sales_act,
'fqn1_sales_act':fqn1_sales_act,
'fqn2_sales_act':fqn2_sales_act,
'fqn3_sales_act':fqn3_sales_act,
'fqn4_sales_act':fqn4_sales_act,
'fqn5_sales_act':fqn5_sales_act,
'fqn6_sales_act':fqn6_sales_act,
'fqn7_sales_act':fqn7_sales_act,
'fqn8_sales_act':fqn8_sales_act,


#EPS Guidance        
'guid_eps_q1_high':guid_eps_q1_high,
'guid_eps_q0_high':guid_eps_q0_high,
'guid_eps_qn1_high':guid_eps_qn1_high,
'guid_eps_qn2_high':guid_eps_qn2_high,
'guid_eps_qn3_high':guid_eps_qn3_high,
'guid_eps_qn4_high':guid_eps_qn4_high,
'guid_eps_qn5_high':guid_eps_qn5_high,
'guid_eps_qn6_high':guid_eps_qn6_high,
'guid_eps_qn7_high':guid_eps_qn7_high,
'guid_eps_qn8_high':guid_eps_qn8_high,

'guid_eps_q1_low':guid_eps_q1_low,
'guid_eps_q0_low':guid_eps_q0_low,
'guid_eps_qn1_low':guid_eps_qn1_low,
'guid_eps_qn2_low':guid_eps_qn2_low,
'guid_eps_qn3_low':guid_eps_qn3_low,
'guid_eps_qn4_low':guid_eps_qn4_low,
'guid_eps_qn5_low':guid_eps_qn5_low,
'guid_eps_qn6_low':guid_eps_qn6_low,
'guid_eps_qn7_low':guid_eps_qn7_low,
'guid_eps_qn8_low':guid_eps_qn8_low,
        
#Sales Guidance        
'guid_sales_q1_high':guid_sales_q1_high,
'guid_sales_q0_high':guid_sales_q0_high,
'guid_sales_qn1_high':guid_sales_qn1_high,
'guid_sales_qn2_high':guid_sales_qn2_high,
'guid_sales_qn3_high':guid_sales_qn3_high,
'guid_sales_qn4_high':guid_sales_qn4_high,
'guid_sales_qn5_high':guid_sales_qn5_high,
'guid_sales_qn6_high':guid_sales_qn6_high,
'guid_sales_qn7_high':guid_sales_qn7_high,
'guid_sales_qn8_high':guid_sales_qn8_high,

'guid_sales_q1_low':guid_sales_q1_low,
'guid_sales_q0_low':guid_sales_q0_low,
'guid_sales_qn1_low':guid_sales_qn1_low,
'guid_sales_qn2_low':guid_sales_qn2_low,
'guid_sales_qn3_low':guid_sales_qn3_low,
'guid_sales_qn4_low':guid_sales_qn4_low,
'guid_sales_qn5_low':guid_sales_qn5_low,
'guid_sales_qn6_low':guid_sales_qn6_low,
'guid_sales_qn7_low':guid_sales_qn7_low,
'guid_sales_qn8_low':guid_sales_qn8_low,
      
#Sales, Cash Flow & EPS Consensus Estimates
# 'fq1_eps_cons':fq1_eps_cons,
# 'fq0_eps_cons':fq0_eps_cons,
# 'fqn1_eps_cons':fqn1_eps_cons,
# 'fqn2_eps_cons':fqn2_eps_cons,
# 'fqn3_eps_cons':fqn3_eps_cons,
# 'fqn4_eps_cons':fqn4_eps_cons,
# 'fqn5_eps_cons':fqn5_eps_cons,
# 'fqn6_eps_cons':fqn6_eps_cons,
# 'fqn7_eps_cons':fqn7_eps_cons,
# 'fqn8_eps_cons':fqn8_eps_cons,

'fq1_sales_cons':fq1_sales_cons,
'fq0_sales_cons':fq0_sales_cons,
'fqn1_sales_cons':fqn1_sales_cons,
'fqn2_sales_cons':fqn2_sales_cons,
'fqn3_sales_cons':fqn3_sales_cons,
'fqn4_sales_cons':fqn4_sales_cons,
'fqn5_sales_cons':fqn5_sales_cons,
'fqn6_sales_cons':fqn6_sales_cons,
'fqn7_sales_cons':fqn7_sales_cons,
'fqn8_sales_cons':fqn8_sales_cons,
        
        'guidance_days_next': guidance_days_next,
        'guidance_days_prev': guidance_days_prev,


    },
    screen=base_universe
)

output = run_pipeline(pipe_my_alpha_factor, '2015-06-01', '2018-09-01')

print output.head(5)


In [ ]:
#Sales Guidance - high/low average (backstopped)
output['fq1_sales_guid'] = ((output['guid_sales_q1_low'] + output['guid_sales_q1_high'])/2).fillna(output['guid_sales_q1_high']).fillna(output['guid_sales_q1_low'])  
output['fq0_sales_guid'] = ((output['guid_sales_q0_low'] + output['guid_sales_q0_high'])/2).fillna(output['guid_sales_q0_high']).fillna(output['guid_sales_q0_low'])  
output['fqn1_sales_guid'] = ((output['guid_sales_qn1_low'] + output['guid_sales_qn1_high'])/2).fillna(output['guid_sales_qn1_high']).fillna(output['guid_sales_qn1_low'])  
output['fqn2_sales_guid'] = ((output['guid_sales_qn2_low'] + output['guid_sales_qn2_high'])/2).fillna(output['guid_sales_qn2_high']).fillna(output['guid_sales_qn2_low'])  
output['fqn3_sales_guid'] = ((output['guid_sales_qn3_low'] + output['guid_sales_qn3_high'])/2).fillna(output['guid_sales_qn3_high']).fillna(output['guid_sales_qn3_low'])  
output['fqn4_sales_guid'] = ((output['guid_sales_qn4_low'] + output['guid_sales_qn4_high'])/2).fillna(output['guid_sales_qn4_high']).fillna(output['guid_sales_qn4_low'])  
output['fqn5_sales_guid'] = ((output['guid_sales_qn5_low'] + output['guid_sales_qn5_high'])/2).fillna(output['guid_sales_qn5_high']).fillna(output['guid_sales_qn5_low'])  
output['fqn6_sales_guid'] = ((output['guid_sales_qn6_low'] + output['guid_sales_qn6_high'])/2).fillna(output['guid_sales_qn6_high']).fillna(output['guid_sales_qn6_low'])  
output['fqn7_sales_guid'] = ((output['guid_sales_qn7_low'] + output['guid_sales_qn7_high'])/2).fillna(output['guid_sales_qn7_high']).fillna(output['guid_sales_qn7_low'])  
output['fqn8_sales_guid'] = ((output['guid_sales_qn8_low'] + output['guid_sales_qn8_high'])/2).fillna(output['guid_sales_qn8_high']).fillna(output['guid_sales_qn8_low'])  

#CFPS Guidance - high/low average (backstopped)
# output['fq1_cfps_guid'] = ((output['guid_cfps_q1_low'] + output['guid_cfps_q1_high'])/2).fillna(output['guid_cfps_q1_high']).fillna(output['guid_cfps_q1_low'])  
# output['fq0_cfps_guid'] = ((output['guid_cfps_q0_low'] + output['guid_cfps_q0_high'])/2).fillna(output['guid_cfps_q0_high']).fillna(output['guid_cfps_q0_low'])  
# output['fqn1_cfps_guid'] = ((output['guid_cfps_qn1_low'] + output['guid_cfps_qn1_high'])/2).fillna(output['guid_cfps_qn1_high']).fillna(output['guid_cfps_qn1_low'])  
# output['fqn2_cfps_guid'] = ((output['guid_cfps_qn2_low'] + output['guid_cfps_qn2_high'])/2).fillna(output['guid_cfps_qn2_high']).fillna(output['guid_cfps_qn2_low'])  
# output['fqn3_cfps_guid'] = ((output['guid_cfps_qn3_low'] + output['guid_cfps_qn3_high'])/2).fillna(output['guid_cfps_qn3_high']).fillna(output['guid_cfps_qn3_low'])  
# output['fqn4_cfps_guid'] = ((output['guid_cfps_qn4_low'] + output['guid_cfps_qn4_high'])/2).fillna(output['guid_cfps_qn4_high']).fillna(output['guid_cfps_qn4_low'])  
# output['fqn5_cfps_guid'] = ((output['guid_cfps_qn5_low'] + output['guid_cfps_qn5_high'])/2).fillna(output['guid_cfps_qn5_high']).fillna(output['guid_cfps_qn5_low'])  
# output['fqn6_cfps_guid'] = ((output['guid_cfps_qn6_low'] + output['guid_cfps_qn6_high'])/2).fillna(output['guid_cfps_qn6_high']).fillna(output['guid_cfps_qn6_low'])  
# output['fqn7_cfps_guid'] = ((output['guid_cfps_qn7_low'] + output['guid_cfps_qn7_high'])/2).fillna(output['guid_cfps_qn7_high']).fillna(output['guid_cfps_qn7_low'])  
# output['fqn8_cfps_guid'] = ((output['guid_cfps_qn8_low'] + output['guid_cfps_qn8_high'])/2).fillna(output['guid_cfps_qn8_high']).fillna(output['guid_cfps_qn8_low'])  

#EPS Guidance - high/low average (backstopped)
output['fq1_eps_guid'] = ((output['guid_eps_q1_low'] + output['guid_eps_q1_high'])/2).fillna(output['guid_eps_q1_high']).fillna(output['guid_eps_q1_low'])  
output['fq0_eps_guid'] = ((output['guid_eps_q0_low'] + output['guid_eps_q0_high'])/2).fillna(output['guid_eps_q0_high']).fillna(output['guid_eps_q0_low'])  
output['fqn1_eps_guid'] = ((output['guid_eps_qn1_low'] + output['guid_eps_qn1_high'])/2).fillna(output['guid_eps_qn1_high']).fillna(output['guid_eps_qn1_low'])  
output['fqn2_eps_guid'] = ((output['guid_eps_qn2_low'] + output['guid_eps_qn2_high'])/2).fillna(output['guid_eps_qn2_high']).fillna(output['guid_eps_qn2_low'])  
output['fqn3_eps_guid'] = ((output['guid_eps_qn3_low'] + output['guid_eps_qn3_high'])/2).fillna(output['guid_eps_qn3_high']).fillna(output['guid_eps_qn3_low'])  
output['fqn4_eps_guid'] = ((output['guid_eps_qn4_low'] + output['guid_eps_qn4_high'])/2).fillna(output['guid_eps_qn4_high']).fillna(output['guid_eps_qn4_low'])  
output['fqn5_eps_guid'] = ((output['guid_eps_qn5_low'] + output['guid_eps_qn5_high'])/2).fillna(output['guid_eps_qn5_high']).fillna(output['guid_eps_qn5_low'])  
output['fqn6_eps_guid'] = ((output['guid_eps_qn6_low'] + output['guid_eps_qn6_high'])/2).fillna(output['guid_eps_qn6_high']).fillna(output['guid_eps_qn6_low'])  
output['fqn7_eps_guid'] = ((output['guid_eps_qn7_low'] + output['guid_eps_qn7_high'])/2).fillna(output['guid_eps_qn7_high']).fillna(output['guid_eps_qn7_low'])  
output['fqn8_eps_guid'] = ((output['guid_eps_qn8_low'] + output['guid_eps_qn8_high'])/2).fillna(output['guid_eps_qn8_high']).fillna(output['guid_eps_qn8_low'])  

#Sales surprise over guidance --- oops!!! eps guid high was used
output['guid_to_act_sales_surprise_q0'] = output['fq0_sales_act'] - output['guid_sales_q0_high'] 
output['guid_to_act_sales_surprise_qn1'] = output['fqn1_sales_act'] - output['guid_sales_qn1_high']
output['guid_to_act_sales_surprise_qn2'] = output['fqn2_sales_act'] - output['guid_sales_qn2_high']
output['guid_to_act_sales_surprise_qn3'] = output['fqn3_sales_act'] - output['guid_sales_qn3_high']
output['guid_to_act_sales_surprise_qn4'] = output['fqn4_sales_act'] - output['guid_sales_qn4_high']
output['guid_to_act_sales_surprise_qn5'] = output['fqn5_sales_act'] - output['guid_sales_qn5_high']
output['guid_to_act_sales_surprise_qn6'] = output['fqn6_sales_act'] - output['guid_sales_qn6_high']
output['guid_to_act_sales_surprise_qn7'] = output['fqn7_sales_act'] - output['guid_sales_qn7_high']
output['guid_to_act_sales_surprise_qn8'] = output['fqn8_sales_act'] - output['guid_sales_qn8_high']

#Consensus to Guidance Sales Surprise 
output['cons_to_guid_sales_surprise_q1'] = output['fq1_sales_guid'] - output['fq1_sales_cons'] 
output['cons_to_guid_sales_surprise_q0'] = output['fq0_sales_guid'] - output['fq0_sales_cons'] 
output['cons_to_guid_sales_surprise_qn1'] = output['fqn1_sales_guid'] - output['fqn1_sales_cons']
output['cons_to_guid_sales_surprise_qn2'] = output['fqn2_sales_guid'] - output['fqn2_sales_cons']
output['cons_to_guid_sales_surprise_qn3'] = output['fqn3_sales_guid'] - output['fqn3_sales_cons']
output['cons_to_guid_sales_surprise_qn4'] = output['fqn4_sales_guid'] - output['fqn4_sales_cons']
output['cons_to_guid_sales_surprise_qn5'] = output['fqn5_sales_guid'] - output['fqn5_sales_cons']
output['cons_to_guid_sales_surprise_qn6'] = output['fqn6_sales_guid'] - output['fqn6_sales_cons']
output['cons_to_guid_sales_surprise_qn7'] = output['fqn7_sales_guid'] - output['fqn7_sales_cons']
output['cons_to_guid_sales_surprise_qn8'] = output['fqn8_sales_guid'] - output['fqn8_sales_cons']


#GROWTH
#Consensus to Guidance Sales Growth - Works
output['cons_to_guid_sales_growth_q0'] = output['fq1_sales_guid']/output['fq0_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn1'] = output['fq0_sales_guid']/output['fqn1_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn2'] = output['fqn1_sales_guid']/output['fqn2_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn3'] = output['fqn2_sales_guid']/output['fqn3_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn4'] = output['fqn3_sales_guid']/output['fqn4_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn5'] = output['fqn4_sales_guid']/output['fqn5_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn6'] = output['fqn5_sales_guid']/output['fqn6_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn7'] = output['fqn6_sales_guid']/output['fqn7_sales_cons'] - 1
output['cons_to_guid_sales_growth_qn8'] = output['fqn7_sales_guid']/output['fqn8_sales_cons'] - 1


In [ ]:
#Delete unused columns 
columns = ['fq1_sales_cons',
                'fq0_sales_cons',
                'fqn1_sales_cons',
                'fqn2_sales_cons',
                'fqn3_sales_cons',
                'fqn4_sales_cons',
                'fqn5_sales_cons',
                'fqn6_sales_cons',
                'fqn7_sales_cons',
                'fqn8_sales_cons',


'guid_sales_q1_high',
'guid_sales_q0_high',
'guid_sales_qn1_high',
'guid_sales_qn2_high',
'guid_sales_qn3_high',
'guid_sales_qn4_high',
'guid_sales_qn5_high',
'guid_sales_qn6_high',
'guid_sales_qn7_high',
'guid_sales_qn8_high',

'guid_sales_q1_low',
'guid_sales_q0_low',
'guid_sales_qn1_low',
'guid_sales_qn2_low',
'guid_sales_qn3_low',
'guid_sales_qn4_low',
'guid_sales_qn5_low',
'guid_sales_qn6_low',
'guid_sales_qn7_low',
'guid_sales_qn8_low',

'guid_eps_q1_high',
'guid_eps_q0_high',
'guid_eps_qn1_high',
'guid_eps_qn2_high',
'guid_eps_qn3_high',
'guid_eps_qn4_high',
'guid_eps_qn5_high',
'guid_eps_qn6_high',
'guid_eps_qn7_high',
'guid_eps_qn8_high',

'guid_eps_q1_low',
'guid_eps_q0_low',
'guid_eps_qn1_low',
'guid_eps_qn2_low',
'guid_eps_qn3_low',
'guid_eps_qn4_low',
'guid_eps_qn5_low',
'guid_eps_qn6_low',
'guid_eps_qn7_low',
'guid_eps_qn8_low']

output.drop(columns, axis=1, inplace=True)


In [ ]:
#Enterprise value for Quarter beginning
# fq0_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=1)
# fqn1_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=1*66)
# fqn2_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=2*66)
# fqn3_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=3*66)
# fqn4_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=4*66)
# fqn5_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=5*66)
# fqn6_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=6*66)
# fqn7_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=7*66)
# fqn8_ev = Previous(inputs=[Fundamentals.enterprise_value], window_length=8*66)


# fq0_cp = Previous(inputs=[USEquityPricing.close], window_length=1)
# fqn1_cp = Previous(inputs=[USEquityPricing.close], window_length=1*66)
# fqn2_cp = Previous(inputs=[USEquityPricing.close], window_length=2*66)
# fqn3_cp = Previous(inputs=[USEquityPricing.close], window_length=3*66)
# fqn4_cp = Previous(inputs=[USEquityPricing.close], window_length=4*66)
# fqn5_cp = Previous(inputs=[USEquityPricing.close], window_length=5*66)
# fqn6_cp = Previous(inputs=[USEquityPricing.close], window_length=6*66)
# fqn7_cp = Previous(inputs=[USEquityPricing.close], window_length=7*66)
# fqn8_cp = Previous(inputs=[USEquityPricing.close], window_length=8*66)

mcap = MarketCap()

fq0_mcap = Previous([mcap], window_length=1)
fqn1_mcap = Previous([mcap], window_length=1*66)
fqn2_mcap = Previous([mcap], window_length=2*66)
fqn3_mcap = Previous([mcap], window_length=3*66)
fqn4_mcap = Previous([mcap], window_length=4*66)
fqn5_mcap = Previous([mcap], window_length=5*66)
fqn6_mcap = Previous([mcap], window_length=6*66)
fqn7_mcap = Previous([mcap], window_length=7*66)
fqn8_mcap = Previous([mcap], window_length=8*66)

fq0_shares = Previous([Fundamentals.shares_outstanding], window_length=1)
fqn1_shares = Previous([Fundamentals.shares_outstanding], window_length=1*66)
fqn2_shares = Previous([Fundamentals.shares_outstanding], window_length=2*66)
fqn3_shares = Previous([Fundamentals.shares_outstanding], window_length=3*66)
fqn4_shares = Previous([Fundamentals.shares_outstanding], window_length=4*66)
fqn5_shares = Previous([Fundamentals.shares_outstanding], window_length=5*66)
fqn6_shares = Previous([Fundamentals.shares_outstanding], window_length=6*66)
fqn7_shares = Previous([Fundamentals.shares_outstanding], window_length=7*66)
fqn8_shares = Previous([Fundamentals.shares_outstanding], window_length=8*66)


#Generate Lagged Enterprise Value
pipe_ev = Pipeline(
    columns={
#         'fq0_ev':fq0_ev,
#                 'fqn1_ev':fqn1_ev,
#                 'fqn2_ev':fqn2_ev,
#                 'fqn3_ev':fqn3_ev,
#                 'fqn4_ev':fqn4_ev,
#                 'fqn5_ev':fqn5_ev,
#                 'fqn6_ev':fqn6_ev,
#                 'fqn7_ev':fqn7_ev,
#                 'fqn8_ev':fqn8_ev,
        
#         'fq0_cp':fq0_cp,
# 'fqn1_cp':fqn1_cp,
# 'fqn2_cp':fqn2_cp,
# 'fqn3_cp':fqn3_cp,
# 'fqn4_cp':fqn4_cp,
# 'fqn5_cp':fqn5_cp,
# 'fqn6_cp':fqn6_cp,
# 'fqn7_cp':fqn7_cp,
# 'fqn8_cp':fqn8_cp,
        
        'fq0_mcap':fq0_mcap,
'fqn1_mcap':fqn1_mcap,
'fqn2_mcap':fqn2_mcap,
'fqn3_mcap':fqn3_mcap,
'fqn4_mcap':fqn4_mcap,
'fqn5_mcap':fqn5_mcap,
'fqn6_mcap':fqn6_mcap,
'fqn7_mcap':fqn7_mcap,
'fqn8_mcap':fqn8_mcap,
        
'fq0_shares':fq0_shares,
'fqn1_shares':fqn1_shares,
'fqn2_shares':fqn2_shares,
'fqn3_shares':fqn3_shares,
'fqn4_shares':fqn4_shares,
'fqn5_shares':fqn5_shares,
'fqn6_shares':fqn6_shares,
'fqn7_shares':fqn7_shares,
'fqn8_shares':fqn8_shares

        
  },
    screen=base_universe
)

output_n = run_pipeline(pipe_ev, '2015-06-01', '2018-09-01')

print output_n.head(5)

In [ ]:
output['guid_sales_to_mcap_q1'] = output['fq1_sales_guid']/output_n['fq0_mcap']
output['guid_sales_to_mcap_q0'] = output['fq0_sales_guid']/output_n['fqn1_mcap']
output['guid_sales_to_mcap_qn1'] = output['fqn1_sales_guid']/output_n['fqn2_mcap']
output['guid_sales_to_mcap_qn2'] = output['fqn2_sales_guid']/output_n['fqn3_mcap']
output['guid_sales_to_mcap_qn3'] = output['fqn3_sales_guid']/output_n['fqn4_mcap']
output['guid_sales_to_mcap_qn4'] = output['fqn4_sales_guid']/output_n['fqn5_mcap']
output['guid_sales_to_mcap_qn5'] = output['fqn5_sales_guid']/output_n['fqn6_mcap']
output['guid_sales_to_mcap_qn6'] = output['fqn6_sales_guid']/output_n['fqn7_mcap']
output['guid_sales_to_mcap_qn7'] = output['fqn7_sales_guid']/output_n['fqn8_mcap']

# columns=[ 'fq0_mcap',
# 'fqn1_mcap',
# 'fqn2_mcap',
# 'fqn3_mcap',
# 'fqn4_mcap',
# 'fqn5_mcap',
# 'fqn6_mcap',
# 'fqn7_mcap',
# 'fqn8_mcap']

# output_n.drop(columns, inplace=True, axis=1)


output['fq1_cost_guid'] = output['fq1_sales_guid'] - output['fq1_eps_guid']*output_n['fq0_shares']
output['fq0_cost_guid'] = output['fq0_sales_guid'] - output['fq0_eps_guid']*output_n['fqn1_shares']
output['fqn1_cost_guid'] = output['fqn1_sales_guid'] - output['fqn1_eps_guid']*output_n['fqn2_shares']
output['fqn2_cost_guid'] = output['fqn2_sales_guid'] - output['fqn2_eps_guid']*output_n['fqn3_shares']
output['fqn3_cost_guid'] = output['fqn3_sales_guid'] - output['fqn3_eps_guid']*output_n['fqn4_shares']
output['fqn4_cost_guid'] = output['fqn4_sales_guid'] - output['fqn4_eps_guid']*output_n['fqn5_shares']
output['fqn5_cost_guid'] = output['fqn5_sales_guid'] - output['fqn5_eps_guid']*output_n['fqn6_shares']
output['fqn6_cost_guid'] = output['fqn6_sales_guid'] - output['fqn6_eps_guid']*output_n['fqn7_shares']
output['fqn7_cost_guid'] = output['fqn7_sales_guid'] - output['fqn7_eps_guid']*output_n['fqn8_shares']


# print output.head(2)

In [ ]:
#Revenue Efficiency
output['fq1_sales_to_cost_guid'] = output['fq1_sales_guid']/output['fq1_cost_guid']
output['fq0_sales_to_cost_guid'] = output['fq0_sales_guid']/output['fq0_cost_guid']
output['fqn1_sales_to_cost_guid'] = output['fqn1_sales_guid']/output['fqn1_cost_guid']
output['fqn2_sales_to_cost_guid'] = output['fqn2_sales_guid']/output['fqn2_cost_guid']
output['fqn3_sales_to_cost_guid'] = output['fqn3_sales_guid']/output['fqn3_cost_guid']
output['fqn4_sales_to_cost_guid'] = output['fqn4_sales_guid']/output['fqn4_cost_guid']
output['fqn5_sales_to_cost_guid'] = output['fqn5_sales_guid']/output['fqn5_cost_guid']
output['fqn6_sales_to_cost_guid'] = output['fqn6_sales_guid']/output['fqn6_cost_guid']
output['fqn7_sales_to_cost_guid'] = output['fqn7_sales_guid']/output['fqn7_cost_guid']


In [ ]:
alpha1 = signalize(-output[[
            'fq1_eps_guid',
            'fq0_eps_guid',
            'fqn1_eps_guid',
            'fqn2_eps_guid',
            'fqn3_eps_guid',
            'fqn4_eps_guid',
            'fqn5_eps_guid',
            'fqn6_eps_guid',
            'fqn7_eps_guid']].std(axis=1))


output['flag'] = 1.0 
output.loc[output['guidance_days_prev'] < 5, 'flag'] = 0.0
output.loc[output['guidance_days_prev'] > 40, 'flag'] = 0.0

alpha11_5_40 = signalize(-output[[
            'fq1_eps_guid',
            'fq0_eps_guid',
            'fqn1_eps_guid',
            'fqn2_eps_guid',
            'fqn3_eps_guid',
            'fqn4_eps_guid',
            'fqn5_eps_guid',
            'fqn6_eps_guid',
            'fqn7_eps_guid']].std(axis=1) * output['flag'])


output['flag'] = 0.0 
output.loc[output['guidance_days_prev'] < 5, 'flag'] = 1.0
output.loc[output['guidance_days_prev'] > 40, 'flag'] = 1.0

alpha11_5_40_out = signalize(-output[[
            'fq1_eps_guid',
            'fq0_eps_guid',
            'fqn1_eps_guid',
            'fqn2_eps_guid',
            'fqn3_eps_guid',
            'fqn4_eps_guid',
            'fqn5_eps_guid',
            'fqn6_eps_guid',
            'fqn7_eps_guid']].std(axis=1) * output['flag'])




In [ ]:
# W
alpha1 = signalize(-output[[
            'fq1_eps_guid',
            'fq0_eps_guid',
            'fqn1_eps_guid',
            'fqn2_eps_guid',
            'fqn3_eps_guid',
            'fqn4_eps_guid',
            'fqn5_eps_guid',
            'fqn6_eps_guid',
            'fqn7_eps_guid']].std(axis=1))

# guid_to_act_sales_surprise_std = output[[
#     'guid_to_act_sales_surprise_q0',
#     'guid_to_act_sales_surprise_qn1', 
#     'guid_to_act_sales_surprise_qn2',
#     'guid_to_act_sales_surprise_qn3',
#     'guid_to_act_sales_surprise_qn4',
#     'guid_to_act_sales_surprise_qn5',
#     'guid_to_act_sales_surprise_qn6',
#     'guid_to_act_sales_surprise_qn7',
#     'guid_to_act_sales_surprise_qn8'
# ]].std(axis=1)

# guid_to_act_sales_surprise_mean = output[[
#     'guid_to_act_sales_surprise_q0',
#     'guid_to_act_sales_surprise_qn1', 
#     'guid_to_act_sales_surprise_qn2',
#     'guid_to_act_sales_surprise_qn3',
#     'guid_to_act_sales_surprise_qn4',
#     'guid_to_act_sales_surprise_qn5',
#     'guid_to_act_sales_surprise_qn6',
#     'guid_to_act_sales_surprise_qn7',
#     'guid_to_act_sales_surprise_qn8'
# ]].mean(axis=1)

#DW
# alpha39 = signalize(guid_to_act_sales_surprise_mean/guid_to_act_sales_surprise_std)

alpha21 = signalize((output['fq1_sales_guid'] - output['fq0_sales_guid'])/output['fq0_sales_guid'])
alpha22 = signalize((output['fq1_sales_guid'] - output['fqn1_sales_guid'])/output['fqn1_sales_guid'])
alpha23 = signalize((output['fq1_sales_guid'] - output['fqn2_sales_guid'])/output['fqn2_sales_guid'])

alpha2 = (alpha21 + alpha22 + alpha23)/3

cs_cons_to_guid_sales_surprise_mean = output[['cons_to_guid_sales_surprise_q1',
                                    'cons_to_guid_sales_surprise_q0',
                                    'cons_to_guid_sales_surprise_qn1',
                                    'cons_to_guid_sales_surprise_qn2',
                                    'cons_to_guid_sales_surprise_qn3',
                                    'cons_to_guid_sales_surprise_qn4',
                                    'cons_to_guid_sales_surprise_qn5',
                                    'cons_to_guid_sales_surprise_qn6',
                                    'cons_to_guid_sales_surprise_qn7',
                                         ]].mean(axis=1)

cs_cons_to_guid_sales_surprise_std = output[['cons_to_guid_sales_surprise_q1',
                                    'cons_to_guid_sales_surprise_q0',
                                    'cons_to_guid_sales_surprise_qn1',
                                    'cons_to_guid_sales_surprise_qn2',
                                    'cons_to_guid_sales_surprise_qn3',
                                    'cons_to_guid_sales_surprise_qn4',
                                    'cons_to_guid_sales_surprise_qn5',
                                    'cons_to_guid_sales_surprise_qn6',
                                    'cons_to_guid_sales_surprise_qn7',
                                         ]].std(axis=1)


alpha3 = signalize(-cs_cons_to_guid_sales_surprise_std)

y = 1/output[['guid_sales_to_mcap_q1', 
'guid_sales_to_mcap_q0',
'guid_sales_to_mcap_qn1',
'guid_sales_to_mcap_qn2',
'guid_sales_to_mcap_qn3',
'guid_sales_to_mcap_qn4',
'guid_sales_to_mcap_qn5',
'guid_sales_to_mcap_qn6']]

alpha53 = signalize(y.mean(axis=1))
# alpha54 = signalize(-y.std(axis=1))
# alpha55 = signalize(y.mean(axis=1)/y.std(axis=1))

z = output[['fq1_cost_guid',
            'fq0_cost_guid',
            'fqn1_cost_guid',
            'fqn2_cost_guid',
            'fqn3_cost_guid',
            'fqn4_cost_guid',
            'fqn5_cost_guid',
            'fqn6_cost_guid',
            'fqn7_cost_guid',
           ]]

alpha56 = signalize(-z.std(axis=1))
# alpha57 = signalize(z.mean(axis=1)/z.std(axis=1))


In [ ]:
#Does (Sales Efficiency)
# sales_to_cost = output[['fq1_sales_to_cost_guid',
#             'fq0_sales_to_cost_guid',
#             'fqn1_sales_to_cost_guid',
#             'fqn2_sales_to_cost_guid',
#             'fqn3_sales_to_cost_guid',
#             'fqn4_sales_to_cost_guid',
#             'fqn5_sales_to_cost_guid',
#             'fqn6_sales_to_cost_guid',
#             'fqn7_sales_to_cost_guid',
#            ]]

# sales_to_cost_mean = sales_to_cost.mean(axis=1)
# sales_to_cost_std = sales_to_cost.std(axis=1)

# alpha58 = signalize(-sales_to_cost_std)
# alpha59 = signalize(sales_to_cost_mean/sales_to_cost_std)


In [ ]:
#Guidance to Actual Sales Surprise to Stock's M-cap
output['guid_to_act_sls_surprise_per_share_q0'] = output['guid_to_act_sales_surprise_q0']/output_n['fq0_shares']
output['guid_to_act_sls_surprise_per_share_qn1'] = output['guid_to_act_sales_surprise_qn1']/output_n['fqn1_shares']
output['guid_to_act_sls_surprise_per_share_qn2'] = output['guid_to_act_sales_surprise_qn2']/output_n['fqn2_shares']
output['guid_to_act_sls_surprise_per_share_qn3'] = output['guid_to_act_sales_surprise_qn3']/output_n['fqn3_shares']
output['guid_to_act_sls_surprise_per_share_qn4'] = output['guid_to_act_sales_surprise_qn4']/output_n['fqn4_shares']
output['guid_to_act_sls_surprise_per_share_qn5'] = output['guid_to_act_sales_surprise_qn5']/output_n['fqn5_shares']
output['guid_to_act_sls_surprise_per_share_qn6'] = output['guid_to_act_sales_surprise_qn6']/output_n['fqn6_shares']
output['guid_to_act_sls_surprise_per_share_qn7'] = output['guid_to_act_sales_surprise_qn7']/output_n['fqn7_shares']

columns=[ 'fq0_shares',
 'fqn1_shares',
 'fqn2_shares',
 'fqn3_shares',
 'fqn4_shares',
 'fqn5_shares',
 'fqn6_shares',
 'fqn7_shares',
 'fqn8_shares']

output_n.drop(columns, inplace=True, axis=1)


sls_surprise_per_share_mean = output[['guid_to_act_sls_surprise_per_share_q0',
                           'guid_to_act_sls_surprise_per_share_qn1',
                           'guid_to_act_sls_surprise_per_share_qn2',
                           'guid_to_act_sls_surprise_per_share_qn3',
                           'guid_to_act_sls_surprise_per_share_qn4',
                           'guid_to_act_sls_surprise_per_share_qn5',
                           'guid_to_act_sls_surprise_per_share_qn6',
                           'guid_to_act_sls_surprise_per_share_qn7']].mean(axis=1)

sls_surprise_per_share_std = output[['guid_to_act_sls_surprise_per_share_q0',
                           'guid_to_act_sls_surprise_per_share_qn1',
                           'guid_to_act_sls_surprise_per_share_qn2',
                           'guid_to_act_sls_surprise_per_share_qn3',
                           'guid_to_act_sls_surprise_per_share_qn4',
                           'guid_to_act_sls_surprise_per_share_qn5',
                           'guid_to_act_sls_surprise_per_share_qn6',
                           'guid_to_act_sls_surprise_per_share_qn7']].std(axis=1)
         

alpha60 = signalize(sls_surprise_per_share_mean)    
alpha61 = signalize(-sls_surprise_per_share_std)
alpha62 = signalize(sls_surprise_per_share_mean/sls_surprise_per_share_std)


output['guid_to_act_sls_surprise_to_mcap_q0'] = output['guid_to_act_sales_surprise_q0']/output_n['fq0_mcap']
output['guid_to_act_sls_surprise_to_mcap_qn1'] = output['guid_to_act_sales_surprise_qn1']/output_n['fqn1_mcap']
output['guid_to_act_sls_surprise_to_mcap_qn2'] = output['guid_to_act_sales_surprise_qn2']/output_n['fqn2_mcap']
output['guid_to_act_sls_surprise_to_mcap_qn3'] = output['guid_to_act_sales_surprise_qn3']/output_n['fqn3_mcap']
output['guid_to_act_sls_surprise_to_mcap_qn4'] = output['guid_to_act_sales_surprise_qn4']/output_n['fqn4_mcap']
output['guid_to_act_sls_surprise_to_mcap_qn5'] = output['guid_to_act_sales_surprise_qn5']/output_n['fqn5_mcap']
output['guid_to_act_sls_surprise_to_mcap_qn6'] = output['guid_to_act_sales_surprise_qn6']/output_n['fqn6_mcap']
output['guid_to_act_sls_surprise_to_mcap_qn7'] = output['guid_to_act_sales_surprise_qn7']/output_n['fqn7_mcap']

columns=[ 'fq0_mcap',
 'fqn1_mcap',
 'fqn2_mcap',
 'fqn3_mcap',
 'fqn4_mcap',
 'fqn5_mcap',
 'fqn6_mcap',
 'fqn7_mcap',
 'fqn8_mcap']

output_n.drop(columns, inplace=True, axis=1)


sls_surprise_to_mcap_mean = output[['guid_to_act_sls_surprise_to_mcap_q0',
                           'guid_to_act_sls_surprise_to_mcap_qn1',
                           'guid_to_act_sls_surprise_to_mcap_qn2',
                           'guid_to_act_sls_surprise_to_mcap_qn3',
                           'guid_to_act_sls_surprise_to_mcap_qn4',
                           'guid_to_act_sls_surprise_to_mcap_qn5',
                           'guid_to_act_sls_surprise_to_mcap_qn6',
                           'guid_to_act_sls_surprise_to_mcap_qn7']].mean(axis=1)

sls_surprise_to_mcap_std = output[['guid_to_act_sls_surprise_to_mcap_q0',
                           'guid_to_act_sls_surprise_to_mcap_qn1',
                           'guid_to_act_sls_surprise_to_mcap_qn2',
                           'guid_to_act_sls_surprise_to_mcap_qn3',
                           'guid_to_act_sls_surprise_to_mcap_qn4',
                           'guid_to_act_sls_surprise_to_mcap_qn5',
                           'guid_to_act_sls_surprise_to_mcap_qn6',
                           'guid_to_act_sls_surprise_to_mcap_qn7']].std(axis=1)
         

    
alpha63 = signalize(sls_surprise_to_mcap_mean)
alpha64 = signalize(-sls_surprise_to_mcap_std)
alpha65 = signalize(sls_surprise_to_mcap_mean/sls_surprise_to_mcap_std)
                     

In [ ]:

# print efficiency.head()
# print output[['fq1_sales_guid']].head()

# print output_ev[['fq0_ev']].head()

# z = output['guid_sales_to_ev_q1'] - output['guid_sales_to_ev_q0']
# print z
# print output[['guid_sales_to_ev_q1']] 
# print output[['guid_sales_to_ev_q0']]

# print alpha41.head(5) 
# print y.head(5)
# print 1/y
# p

In [ ]:
print output['guid_to_act_sls_surprise_to_mcap_q0'].corr(1/output['guid_sales_to_mcap_q1'])

In [ ]:
# print alpha41.head()
output['alpha1'] = alpha1
output['alpha11'] = alpha11

output['alpha2'] = alpha2
output['alpha3'] = alpha3

# output['alpha39'] = alpha39

# output['alpha50'] = alpha50
# output['alpha51'] = alpha51
# output['alpha52'] = alpha52

output['alpha53'] = alpha53
# output['alpha54'] = alpha54
# output['alpha55'] = alpha55

output['alpha56'] = alpha56
# output['alpha57'] = alpha57

output['alpha60'] = alpha60
output['alpha61'] = alpha61
output['alpha62'] = alpha62

output['alpha63'] = alpha63
output['alpha64'] = alpha64
output['alpha65'] = alpha65



In [ ]:
def wmean(name):
    output['i'] = 0
#     output['i'][output[name].notnull()] = 1.0
    output.loc[output[name].notnull(), 'i'] = 1.0

    return (output[name]*output['mkt_cap']).sum(level=0)/(output['mkt_cap']*output['i']).sum(level=0)

alphas = pd.DataFrame()

for name in ['alpha1', 'alpha11',
             'alpha2',
             'alpha3',
             
#              'alpha39',
             'alpha53',
             'alpha56',

             'alpha60',
              'alpha61',
             'alpha62',
             
             'alpha63', 'alpha64', 'alpha65'
            ]:
    alphas[name] = output[name].sub(output[name].mean(level=0), level=0)
#     alphas[name] = output[name].sub(wmean(name), level=0)

# for name in [ #'alpha3', 
#              'alpha39',
#              'alpha50',
#              'alpha51',
#              'alpha52',
             
#              'alpha53',
#              'alpha54',
#              'alpha55', 
             
#              'alpha56',
#              'alpha57',
#              'alpha58',
    
#             ]:
# #     alphas[name] = output[name].sub(output[name].mean(level=0), level=0)
#     alphas[name] = output[name].sub(wmean(name), level=0)


In [ ]:
assets = output.index.levels[1].unique()
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
pricing = get_pricing(assets, start_date='2015-06-01', end_date='2018-09-01', fields='close_price')
print "DONE Pricing"


In [ ]:
returns = pricing.pct_change(periods=5).shift(-5)
returns = returns.sub(returns.mean(axis=1), axis=0).stack()
# r_m = returns.mean(level=0)
# print returns

data = pd.concat([returns.rename('returns'), alphas], axis=1).dropna()

data.reset_index(inplace=True)
data = data.rename(columns={'level_0':'dates', 'level_1': 'sid'})
# print data.head()

In [ ]:
# print (alpha1 + alpha2 + alpha3).corr(alpha61 + alpha62 + alpha63)
print (alpha53).corr(alpha56)

In [ ]:
data['d1'] = data['alpha1'] + data['alpha2'] +  data['alpha3'] 
# data['d2'] = data['alpha1'] + data['alpha2'] +  data['alpha51'] 
# data['d3'] = data['alpha2'] + data['alpha3'] +  data['alpha51'] 
# data['d4'] = data['alpha1'] + data['alpha2'] + data['alpha3'] +  data['alpha50']
data['d5'] = data['alpha1'] + data['alpha2'] + data['alpha3'] +  data['alpha53']
data['d6'] = data['alpha1'] + data['alpha2'] + data['alpha3'] +  data['alpha53'] + data['alpha56']
data['d7'] = data['alpha1'] + data['alpha2'] + data['alpha3'] +  data['alpha53'] + data['alpha56'] + data['alpha60']
data['d8'] = data['alpha1'] + data['alpha2'] + data['alpha3'] +  data['alpha53'] + data['alpha56'] + data['alpha61']
data['d9'] = data['alpha1'] + data['alpha2'] +  data['alpha53'] + data['alpha56'] + data['alpha61']



data['d10'] = data['alpha1'] + data['alpha2'] +  data['alpha3'] + data['alpha61'] + data['alpha56']

est = sm.OLS(
    100*data[['returns']], 
#         data[['alpha1', 'alpha2', 'alpha3', 'alpha53', 'alpha56', 'alpha61']])
            data[['alpha11']])
#     data[['d9']])
est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()


In [ ]:
print alpha3.corr(alpha53)

In [ ]:
net_alpha = alpha1 + alpha2 + alpha3
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)